# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# in case of analysis 

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
from config import g_key
import json
import requests
import os
import numpy as np
import gmaps
import time


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:

city_path = "../output/cities.csv"

cities_df = pd.read_csv(city_path)

cities_df = cities_df.drop(columns = ["Unnamed: 0"])
                          
cities_df                       
                          

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hobart,-42.8794,147.3294,14.00,51.0,40.0,7.72,AU,1.619342e+09
1,kharan,28.5833,65.4167,35.90,9.0,0.0,3.95,PK,1.619342e+09
2,itarema,-2.9248,-39.9167,24.17,90.0,89.0,3.39,BR,1.619342e+09
3,bredasdorp,-34.5322,20.0403,15.00,88.0,100.0,3.09,ZA,1.619342e+09
4,ushuaia,-54.8000,-68.3000,8.00,71.0,0.0,5.14,AR,1.619341e+09
...,...,...,...,...,...,...,...,...,...
535,laguna,38.4210,-121.4238,12.22,71.0,1.0,2.57,US,1.619342e+09
536,beboto,8.2668,16.9390,30.11,48.0,99.0,4.05,TD,1.619342e+09
537,belomorsk,64.5232,34.7668,7.97,77.0,43.0,3.06,RU,1.619342e+09
538,batagay-alyta,67.8006,130.4114,0.20,84.0,100.0,2.02,RU,1.619342e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [20]:
# Store the lat and long

locations = cities_df[["Lat", "Lng"]].astype(float)

humid_weight = cities_df["Humidity"].astype(float)

# Create Heatmap for Humidity 

fig = gmaps.figure()

humidity_heat = gmaps.heatmap_layer(locations, weights=humid_weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=8)

fig.add_layer(humidity_heat)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
# reduce dataframe based on temp 
 
toasty_weather_df = cities_df[cities_df["Max Temp"] >30]
toasty_weather2_df = toasty_weather_df[toasty_weather_df["Max Temp"] <40]
no_wind_df = toasty_weather2_df[toasty_weather2_df["Wind Speed"] < 7]
perfect_df = no_wind_df[no_wind_df["Cloudiness"] < 5]
perfect_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,kharan,28.5833,65.4167,35.90,9.0,0.0,3.95,PK,1.619342e+09
40,tura,25.5198,90.2201,37.79,12.0,0.0,2.90,IN,1.619342e+09
46,marzuq,14.4000,46.4667,32.26,16.0,1.0,4.20,YE,1.619342e+09
138,taoudenni,22.6783,-3.9836,33.43,9.0,0.0,1.88,ML,1.619342e+09
196,aguie,13.5060,7.7786,31.00,42.0,0.0,4.63,NE,1.619342e+09
200,agadez,19.7500,10.2500,36.24,8.0,2.0,4.98,NE,1.619342e+09
245,dalbandin,28.8947,64.4101,35.90,4.0,0.0,4.54,PK,1.619342e+09
308,jaisalmer,26.9147,70.9181,37.84,7.0,0.0,0.63,IN,1.619342e+09
349,sharjah,25.3573,55.4033,36.00,19.0,0.0,6.17,AE,1.619342e+09
444,riyadh,24.6877,46.7219,37.00,6.0,0.0,5.14,SA,1.619342e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [33]:
# Create a new column 

perfect_df["Hotel Name"] = ""
hotel_df = perfect_df
hotel_df

/Users/stephaniesalvona/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,kharan,28.5833,65.4167,35.90,9.0,0.0,3.95,PK,1.619342e+09,
40,tura,25.5198,90.2201,37.79,12.0,0.0,2.90,IN,1.619342e+09,
46,marzuq,14.4000,46.4667,32.26,16.0,1.0,4.20,YE,1.619342e+09,
138,taoudenni,22.6783,-3.9836,33.43,9.0,0.0,1.88,ML,1.619342e+09,
196,aguie,13.5060,7.7786,31.00,42.0,0.0,4.63,NE,1.619342e+09,
200,agadez,19.7500,10.2500,36.24,8.0,2.0,4.98,NE,1.619342e+09,
245,dalbandin,28.8947,64.4101,35.90,4.0,0.0,4.54,PK,1.619342e+09,
308,jaisalmer,26.9147,70.9181,37.84,7.0,0.0,0.63,IN,1.619342e+09,
349,sharjah,25.3573,55.4033,36.00,19.0,0.0,6.17,AE,1.619342e+09,
444,riyadh,24.6877,46.7219,37.00,6.0,0.0,5.14,SA,1.619342e+09,


In [44]:
radius = 5000
coord = "28.5833, 65.4167"

params = {
    "location": coord,
    "radius": radius,
    "type": "lodging",
    "key": g_key,
    "keyword": "hotel"
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

response = requests.get(base_url, params=params)
response_json = response.json()
print(json.dumps(response_json, indent=4, sort_keys=True))


{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 28.5837022,
                    "lng": 65.4186936
                },
                "viewport": {
                    "northeast": {
                        "lat": 28.58505472989273,
                        "lng": 65.42008092989272
                    },
                    "southwest": {
                        "lat": 28.58235507010728,
                        "lng": 65.41738127010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Mansoor Hotel & Restaurant",
            "opening_hours": {
                "open_now": false
            },
            "place_id": "ChIJraf22MNfzj4RHxnqr52YX6s",
            "plus_code": {
                "compound_code": "HCM9+FF Kharan, Pakistan

In [52]:
# Find nearest hotels to cities 

# set params 

radius = 5000

params = {
    "radius": radius,
    "types": "lodging",
    "key": g_key,
    "keyword": "hotel"
}


# loop through df for hotels 

for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    params["location"] = f"{latitude},{longitude}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params)
    response_json = response.json()
    
    try:
        hotels_df.loc[index, "Hotel Name"] = response_json["results"][0]["name"]
        print("worked, whers the hotel ")
        
    except:
        print("why?")
        pass

hotel_df


why?
why?
why?
why?


KeyboardInterrupt: 

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
